In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,28602
2,Huelva,Confirmados PDIA 14 días,1092
3,Huelva,Tasa PDIA 14 días,"212,7949802209794"
4,Huelva,Confirmados PDIA 7 días,452
5,Huelva,Total Confirmados,28777
6,Huelva,Curados,24457
7,Huelva,Fallecidos,364


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  28602.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7414.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1273 personas en los últimos 7 días 

Un positivo PCR cada 582 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,28602.0,452.0,1092.0,513170.0,88.079973,212.794980,120.0
Condado-Campiña,9016.0,230.0,568.0,156231.0,147.217902,363.564209,60.0
Huelva-Costa,16664.0,181.0,407.0,289548.0,62.511224,140.563913,49.0
Huelva (capital),7414.0,113.0,247.0,143837.0,78.561149,171.722158,27.0
Almonte,1521.0,68.0,170.0,24507.0,277.471743,693.679357,23.0
Bollullos Par del Condado,821.0,38.0,75.0,14387.0,264.127337,521.303955,12.0
Moguer,1105.0,33.0,79.0,21867.0,150.912334,361.274981,9.0
Aljaraque,1001.0,9.0,19.0,21474.0,41.911148,88.479091,8.0
Sierra de Huelva-Andévalo Central,2547.0,33.0,101.0,67391.0,48.967963,149.871645,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,232.0,17.0,42.0,3261.0,521.312481,1287.948482,1.0
Cumbres Mayores,49.0,1.0,19.0,1749.0,57.175529,1086.335049,0.0
Cortelazor,7.0,2.0,3.0,299.0,668.896321,1003.344482,0.0
Almonte,1521.0,68.0,170.0,24507.0,277.471743,693.679357,23.0
Beas,197.0,4.0,23.0,4341.0,92.144667,529.831836,0.0
Bollullos Par del Condado,821.0,38.0,75.0,14387.0,264.127337,521.303955,12.0
Cañaveral de León,5.0,2.0,2.0,396.0,505.050505,505.050505,0.0
Rociana del Condado,515.0,17.0,40.0,7939.0,214.132762,503.841794,3.0
Bonares,255.0,12.0,26.0,6060.0,198.019802,429.042904,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,49.0,1.0,19.0,1749.0,57.175529,1086.335049,0.0,0.052632
Beas,197.0,4.0,23.0,4341.0,92.144667,529.831836,0.0,0.173913
Minas de Riotinto,112.0,1.0,5.0,3812.0,26.232949,131.164743,1.0,0.200000
Gibraleón,587.0,6.0,23.0,12737.0,47.106854,180.576274,2.0,0.260870
Palos de la Frontera,582.0,7.0,22.0,11742.0,59.615057,187.361608,2.0,0.318182
Cartaya,1269.0,15.0,47.0,20083.0,74.690036,234.028781,4.0,0.319149
Sierra de Huelva-Andévalo Central,2547.0,33.0,101.0,67391.0,48.967963,149.871645,8.0,0.326733
Aroche,90.0,1.0,3.0,3054.0,32.743942,98.231827,0.0,0.333333
San Juan del Puerto,450.0,11.0,32.0,9411.0,116.884497,340.027627,3.0,0.343750
